# Geometry in SIRF

Authors: David Atkinson

First version: 20 June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).
Copyright 2021 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

**Getting Started**

Data for the geometry notebooks should be in the notebook folder `nifti`. The nifti data needs to be downloaded once from Zenodo - see the notebook `get_nifti_data` to download the data.

In [ ]:
# Setup the working directory for the notebook
# (Requires download_data.sh to have been run once before)
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Geometry')

In [ ]:
# Import required packages
import nibabel 
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Data for geometry notebooks when run is ./nifti/*.nii
data_path = os.getcwd()

In [ ]:
# Set numpy print options to print small numbers as zero etc.
np.set_printoptions(precision=3,suppress=True)

### `imshow` and the image coordinate system

In this Notebook, when referring to array elements, we use 'first', 'second' etc with their usual English meaning. In Python, the first array index is 0. To avoid confusion, we do not use the term 'zeroth'.

We use the matplotlib `imshow` command with default options for origin and orientation (this corresponds to MATLAB). `imshow(A)` plots array `A` as an image using the ordering `A[row_number, column_number]`, the first row is at the top of the image, and, pixel centres are at integer positions. The array indices are used as integer image coordinates. The top left pixel centre is at (0,0) when using 0-based Python. 

In [ ]:
# Demo of imshow using array with 2 rows and 3 columns.
# Note the array indices are mapped to integer image coordinates
#  (0,0) is at the centre of the top left pixel

testim = [[0, 0.25, 0],
          [1, 0.5,  0]]
ax = plt.imshow(testim, cmap='gray')

### Placing the image in a 3D Patient Coordinate system

DICOM and SIRF use an LPH+ patient coordinate system. Increasing the value of the first patient coordinate corresponds to moving in the positive Left direction i.e. towards the patient left. Increasing the second moves in the Posterior direction and the third in the Head direction, also known as Superior. (This coordinate system is therefore also often called LPS).

In DICOM, the 3D position of an image pixel can be found using DICOM parameters `ImageOrientationPatient`, `ImagePositionPatient` and `PixelSpacing`. In brief, the 3D position can be found using these terms in a vector addition.

In SIRF, the 3D position of an array element is given by multiplying an affine matrix __A__ by a coordinate formed from the array indices. This is similar to NIfTI except that NIfTI uses an RAH+ patient coordinate system. It is useful to think of the 3D patient coordinate as determined from array indices and not to imply any direction in either the image or 3D space. SIRF emphasises this by naming the method that provides __A__ as `get_index_to_physical_point_matrix()`. 

* Provided the affine matrix __A__ correctly describes the geometry, the image can appear on screen with any orientation.
* The affine matrix should be updated to maintain the correct correspondence if the image data is manipulated, for example after cropping or rotating. 
* It is usually NOT necessary to flip, rotate or permute an image array. An exception would be to get an expected appearance such as radiological presentation (and if you do this, the matrix should also be updated). 

![Geometry](SIRF_geom.png "Geometry")

Schematic showing the patient LPH+ 3D coordinate system axes and an image in that space. This image could have come from a reconstructed slice through a patient. The 3D patient coordinate __V__ of a point in the image (i,j,k) is given by the upper equation. The top left image pixel is here at image coordinate (0,0,0) with corresponding 3D coordinate __Q__. This corresponds to DICOM `ImagePositionPatient` and NIfTI `offset` (after converting from NIfTI RAH+ to LPH+). Here the image is 2D but given a coordinate with 3rd dimension k. For a single slice out of an image volume, k has the value 0 in the slice pointed to by  `offset`. 

Homogeneous coordinates (with an extra dimension) are used so that __A__ can include the translational `offset`.

#### Example using NIfTI
The data for this Notebook was acquired using an arrangement of 3 phantoms. The circular phantom was towards the patient Head position, the bigger bottle towards the patient Right and the smaller (Littler) bottle to the patient Left. Scans were acquired at different image orientations without moving the phantoms. Each scan had 30 slices. DICOM and NIfTI files were created by the scanner.

In [ ]:
# Data files are in a folder clled nifti below the main data folder
# Set the full file name for the data file
fpath  = os.path.join(data_path , 'nifti')
fn_cor = "OBJECT_phantom_T2W_TSE_Cor_14_1.nii" # Coronal volume, 30 slices

ffn = os.path.join(fpath, fn_cor)  # full file name
print("Full file name: ", ffn)

In [ ]:
ns = nibabel.load(ffn)
# To view NIfTI header parameters when read in using nibabel:
# print(ns.header)  or 
# print("NIfTI pixdim:  ",ns.header['pixdim'])

nad = ns.get_fdata()  # NIfTI image array data
print("NIfTI array data shape: ",nad.shape)

In [ ]:
slc = 15 ;

slc_dat = nad[:,:,slc]
plt.imshow(slc_dat, cmap='gray')

As displayed, the image is in a non-standard orientation with the circular phantom 'head' on the image West side and the littler bottle (patient left) towards the South. (We use compass directions to avoid confusion between image or patient 'left' and 'right'). Despite this, the affine matrix is correct as we will see next.

In [ ]:
# nibabel can convert NIfTI header parameters to an affine matrix
A_RAH = ns.affine

NIfTI uses RAH+. To convert to an LPH+ affine matrix, multiply every element of the first two rows of `A_RAH` with -1. 

In [ ]:
RAH2LPH = [[-1, 0, 0, 0 ],
           [ 0,-1, 0, 0 ],
           [ 0, 0, 1, 0 ],
           [ 0, 0, 0, 1 ]]

A_LPH = np.matmul(RAH2LPH,A_RAH)
print(A_LPH)

Note this affine matrix contains information about the pixel sizes (the scaling on the diagonal), the offset (the right column) and the image orientation.

The DICOM representation is useful to understand and is shown in the following figure.

![DICOM_summary.png](DICOM_summary.png)

Next, find the 3D LPH position of the position of array index [0,0,0]. When using `imshow` as above, this is the centre of the top left pixel and corresponds to the DICOM `ImagePositionPatient` (IPP). 
Note, in DICOM, there is usually one IPP for each frame (2D image), but here there is one offset for the whole volume.

In [ ]:
IPP = np.matmul(A_LPH,[0,0,0,1])  

IPP = IPP[0:3]  # remove last element (homogeneous coordinates)

print("ImagePositionPatient (elements are in the order L P H): ",IPP)

The units are mm. The origin is not defined but is at a consistent point in the patient during a scanning session, even if the bed moves. (In DICOM terms, the origin is the same for all scans with the same `FrameOfReferenceUID`.) 

In DICOM, the image orientation is specified by `ImageOrientationPatient` (IOP) which is two, 3-element, unit vectors, the first pointing East and the second South. We can find these from the affine matrix by finding vectors from the top left pixel (IPP) to one pixel to the East and one to the South.

In [ ]:
IPP_1EAST  = np.matmul(A_LPH, [0,1,0,1]) # one pixel 'east' (next column) 
IPP_1SOUTH = np.matmul(A_LPH, [1,0,0,1]) # one pixel 'south' (next row down)

IPP_1EAST  = IPP_1EAST[0:3]    # remove last element 
IPP_1SOUTH = IPP_1SOUTH[0:3]

# The IPP_*s are 3D coordinates. Find the following vectors:
vec_EAST  = IPP_1EAST  - IPP # vector pointing along row to next column
vec_SOUTH = IPP_1SOUTH - IPP # vector pointing down column to next row

spacing_EAST  = np.linalg.norm(vec_EAST) # vector length (pixel spacing)
spacing_SOUTH = np.linalg.norm(vec_SOUTH)

print("spacing_SOUTH (mm):  ",spacing_SOUTH)
print("spacing_EAST (mm) :  ",spacing_EAST)

# Find the unit vectors corresponding to DICOM ImageOrientationPatient
IOP_EAST = np.multiply(vec_EAST,1./spacing_EAST)
print("IOP_EAST:  ",IOP_EAST)

IOP_SOUTH = np.multiply(vec_SOUTH,1./spacing_SOUTH)
print("IOP_SOUTH: ",IOP_SOUTH)

The IOP_EAST is close to [0 0 -1] (in LPH) meaning that the 'East' direction in the image is almost negative Head, i.e. towards the feet.

The IOP_SOUTH is close to [1 0 0] meaning that going down the image points to the patient's Left.

These are correct for the image as displayed above.

#### SIRF ImageData
The above used NIfTI functionality from the `nibabel` package. Within SIRF, NIfTI files can also be read and written using `sirf.Reg`. The code below creates a SIRF Image Data class and shows how to extract information.

In [ ]:
import sirf.Reg as Reg

In [ ]:
s_imd = Reg.ImageData(ffn)        # Load file as SIRF Image Data

In [ ]:
s_array = s_imd.as_array()
print("SIRF array shape: ",s_array.shape)

In [ ]:
plt.imshow(s_array[:,:,slc], cmap='gray')

In [ ]:
# Use the SIRF geometrical_info method
s_geom_info = s_imd.get_geometrical_info()

s_direction_matrix = s_geom_info.get_direction_matrix()
print("SIRF geom_info direction matrix \n", s_direction_matrix, "\n")

print("SIRF geom_info get_index_to_physical_point_matrix()\n", s_geom_info.get_index_to_physical_point_matrix())

print("\n SIRF s_geom_info.get_info()\n",s_geom_info.get_info() )

print("SIRF s_imd.dimensions()   \n", s_imd.dimensions())
print("SIRF s_imd.as_array().shape \n", s_imd.as_array().shape)

The SIRF `get_direction_matrix` returns a 3x3 direction matrix that has information only about orientations. The first column corresponds to unit vector IOP_SOUTH and the second column to IOP_EAST. 

The SIRF `get_index_to_physical_point_matrix()` returns the same matrix as `A_LPH`. This is the same 4x4 matrix as the nibabel NIfTI affine matrix after conversion to LPH. This 4x4 matrix includes direction, spacing and offset information, whereas the SIRF 3x3 direction matrix is only the direction (also called 'orientation') information. 

## SIRF Issues
Correctly handling geometrical information, and associated array dimensions, is challenging. Conventions within SIRF and the engines used by SIRF can lead to confusion. For example, the `sirf.Reg.ImageData` methods `get_dimensions` and `get_voxel_sizes` return the NIfTI header data directly which can be confusing because it has an extra first dimension.


In [ ]:
print("SIRF s_imd.get_dimensions(): ", s_imd.get_dimensions())
print("NIfTI header dim:            ", ns.header['dim'])    

print("\n SIRF s_imd.get_voxel_sizes() \n", s_imd.get_voxel_sizes(), "\n")

You can get the full NIfTI header data using the `print_header()` method.

In [ ]:
s_imd.print_header()

The STIR engine can read NIfTI files but it does not correctly handle oblique images. 

NIfTI and DICOM use a patient coordinate system which cannot be related to physical hardware directions without a knowledge of the patient orientation (e.g. head-first-supine). 

**TIP**

As much as possible, work with original data and trust the affine matrix over `get` helper functions. Offset, spacing and orientations can be determined from the affine matrix as above.